In [1]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.8/486.8 kB 11.7 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.8/61.8 MB 17.7 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.5.4.60
    Uninstalling opencv-python-4.5.4.60:
      Successfully uninstalled opencv-python-4.5.4.60


In [3]:
!pip freeze > requirements.txt  

In [2]:
import os
import shutil
import random
import json
import glob
import torch

from ultralytics import YOLO
from PIL import Image

In [ ]:
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

In [ ]:
input_folder = "/kaggle/input/ui-kits-simple/rico_dataset_v0.1_semantic_annotations/semantic_annotations"
output_image_folder = "/kaggle/working/data/rico/images"
output_json_folder = "/kaggle/working/data/rico/jsons"

os.makedirs(output_image_folder, exist_ok=True)
os.makedirs(output_json_folder, exist_ok=True)

for file_name in os.listdir(input_folder):
    if file_name.endswith(".png") or file_name.endswith(".jpg"):
        shutil.copy(os.path.join(input_folder, file_name), os.path.join(output_image_folder, file_name))
    elif file_name.endswith(".json"):
        shutil.copy(os.path.join(input_folder, file_name), os.path.join(output_json_folder, file_name))


In [ ]:
#bounds => [left, top, right, bottom]

input_path = "/kaggle/working/data/rico/jsons"
output_path = '/kaggle/working/data/rico/txt'

for filename in os.listdir(input_path):
    with open(os.path.join(input_path, filename)) as f:
        data = json.load(f)

        def get_child_components(data, labels=[]):
            if 'children' in list(data.keys()):
                for c in data['children']:

                    label_map = {
                        'Image': 0,
                        'Text': 1,
                        'Text Button': 2,
                        'Input': 3,
                        'Icon': 4,
                        'List Item': 5,
                        'Another': 6,
                        # more mappings if needed
                    }

                    #Convert these bounds to YOLO format

                    width = c['bounds'][2] - c['bounds'][0]
                    height = c['bounds'][3] - c['bounds'][1]

                    x_center = c['bounds'][0] + width/2
                    y_center = c['bounds'][1] + height/2
                    
                    base = data['bounds']
                    
                    image_width = 1440 #int(base[2])
                    image_height = 2560 #int(base[3])
                    
                    x_center_norm = x_center/image_width
                    y_center_norm = y_center/image_height
                    width_norm = width/image_width
                    height_norm = height/image_height

                    component_index = label_map.get(c['componentLabel'], 0)

                    yolo_format = [component_index, x_center_norm, y_center_norm, width_norm, height_norm]

                    labels.append({c['componentLabel']: yolo_format})

                    get_child_components(c, labels)
            return labels

        try:
            os.mkdir(output_path)
        except:
            pass
        
        output_file_path = os.path.join(output_path, filename[:-5] + '.txt')
        
        with open(output_file_path, 'w') as f:
            for i in get_child_components(data):
                for k in i.keys():
                    f.write('{} {} {} {} {}\n'.format(int(i[k][0]), float(i[k][1]), float(i[k][2]), float(i[k][3]), float(i[k][4])))


In [ ]:
"""Finds corresponding png files after distributed txt files"""

# Define the paths to the image and text files
image_path = "/kaggle/working/data/rico/images"
text_path = "/kaggle/working/data/rico/txt"

# Define the output directories for the train, val, and test sets
train_path = "/kaggle/working/data/rico/train"
val_path = "/kaggle/working/data/rico/val"
test_path = "/kaggle/working/data/rico/test"

# Define the ratio of the train and val sets
train_ratio = 0.70
val_ratio = 0.15
test_ratio = 0.15

# Get the list of text files
text_files = sorted(os.listdir(text_path))

# Shuffle the list of text files
random.shuffle(text_files)

# Calculate the number of files for each set
num_train = int(len(text_files) * train_ratio)
num_val = int(len(text_files) * val_ratio)
num_test = int(len(text_files) - num_train - num_val)

# Assign each file to a set
train_files = text_files[:num_train]
val_files = text_files[num_train:num_train+num_val]
test_files = text_files[num_train+num_val:]

# Create the output directories, if exists delete them and create new ones
if os.path.exists(train_path):
    shutil.rmtree(train_path)
os.makedirs(train_path)

if os.path.exists(val_path):
    shutil.rmtree(val_path)
os.makedirs(val_path)

if os.path.exists(test_path):
    shutil.rmtree(test_path)
os.makedirs(test_path)

# Copy train txt files to the train directory, then copy the corresponding png files
for file in train_files:
    shutil.copy(os.path.join(text_path, file), train_path)
    shutil.copy(os.path.join(image_path, file[:-4] + ".png"), train_path)

# Copy val txt files to the val directory, then copy the corresponding png files
for file in val_files:
    shutil.copy(os.path.join(text_path, file), val_path)
    shutil.copy(os.path.join(image_path, file[:-4] + ".png"), val_path)

# Copy test txt files to the test directory, then copy the corresponding png files
for file in test_files:
    shutil.copy(os.path.join(text_path, file), test_path)
    shutil.copy(os.path.join(image_path, file[:-4] + ".png"), test_path)

In [ ]:
# Print the number of files in each set, both text and image files
print("Number of train files: ", len(os.listdir(train_path)))
print("Number of val files: ", len(os.listdir(val_path)))
print("Number of test files: ", len(os.listdir(test_path)))

In [ ]:
# Load a model
# model_path = "/kaggle/input/core-components/model.yaml"

model = YOLO('yolov8n.yaml') # build a new model from YAML
model = YOLO('yolov8n.pt',task='detect')  # load a pretrained model (recommended for training)
# model = YOLO(yaml_path, task='detect').load('yolov8n.pt') # build from YAML and transfer weights

In [ ]:
# Train the model
yaml_path = "/kaggle/input/core-components/data.yaml"
device = 'cuda:0'  # Use the first available GPU
model.train(data=yaml_path, epochs=2, imgsz=640, device=device, val=True)

In [ ]:
# Export the model as torchscript
best_weight_path = '/kaggle/working/runs/detect/train/weights/best.pt'
model = YOLO(best_weight_path) 
model.fuse()  
model.info(verbose=True)  # Print model information
model.export()

In [ ]:
# Test model with a random screenshot
dir_path = "/kaggle/input/ui-kits-simple/unique_uis/combined"
jpg_files = glob.glob(dir_path + "/*.jpg")

if jpg_files:
    try:
        random_jpg = random.choice(jpg_files)
        image = Image.open(random_jpg)
        result = model.predict(source=image, save=True)
        print(result)
        
    except Exception as e:
        print("Error occurred while processing image:", e)
        
else:
    print("No JPG files found in directory.")

In [ ]:
# import shutil, os

# def kaggle_export(output_dir, zip_name):
#     """Create a zip file and export your work in the kaggle working directory.

#     Args:
#         output_dir (str): The path to the directory to be zipped.
#         zip_name (str): The name of the zip file to be created.

#     Returns:
#         None
#     """
#     if not os.path.exists(output_dir):
#         print(f"[Error]: output directory '{output_dir}' does not exist")
    
#     else:
#         try:
#             shutil.make_archive(zip_name, 'zip', output_dir)
#             print(f"[Success]: Your zip file is successfully created!")

#         except Exception as e:
#             print(f"[Error]: An error occurred while creating zip: {e}")

# kaggle_export("/kaggle/working/runs/detect/train", "model_result")